In [ ]:
# Install packages
%pip install kaggle boto3 python-dotenv
# Imports
import pandas as pd
import os
import kaggle
import zipfile
from mappings import state_province_to_country, country_full_names, country_code_mapping
import boto3
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
API_ACCESS_KEY=os.getenv("API_ACCESS_KEY")
API_SECRET_ACCESS_KEY=os.getenv("API_SECRET_ACCESS_KEY")

# Download and extract dataset
!kaggle datasets download -d camnugent/ufo-sightings-around-the-world

with zipfile.ZipFile('ufo-sightings-around-the-world.zip', 'r') as file:
    file.extractall()

ufo = pd.read_csv('ufo_sighting_data.csv')

ufo['Date_time'] = pd.to_datetime(ufo['Date_time'], format='%m/%d/%Y %H:%M', errors='coerce')
# Drop rows missing latitude or longitude values or date
ufo.dropna(subset=['Date_time', 'latitude', 'longitude'], inplace=True)

missing_country = ufo['country'].isnull().sum()
print(f"Number of missing values in 'country': {missing_country}")
# There are 9670 missing values in the country column.  
total_non_null = ufo['country'].count()
print(f"Total number of non-null vlaues in 'country': {total_non_null}")

# When I explored the data, I found a lot of country names missing in cases where the state or province is present. 
# Let's use state/province to fill in missing fields in the country column.
def fill_countries_from_state(ufo):
    for index, row in ufo.iterrows():
       if pd.isnull(row['country']):
        state_province = row['state/province']
        country = state_province_to_country.get(state_province, None)
        if country:
            ufo.at[index, 'country'] = country
    return ufo

# Check if there are any remaining missing values in the "country" column
ufo=fill_countries_from_state(ufo)
missing_country = ufo['country'].isnull().sum()
print(f"Number of missing values in 'country' after imputation: {missing_country}")


# Initialize AWS Location Service 
client = boto3.client('location',
                      region_name='us-east-1',
                      aws_access_key_id=API_ACCESS_KEY,
                      aws_secret_access_key=API_SECRET_ACCESS_KEY)

def get_full_country_name(lat, lon):
    try:
        print(f"Processing lat: {lat}, lon: {lon}")
        lat_float = float(lat)
        lon_float = float(lon)
        response = client.search_place_index_for_position(
            IndexName='UFO-Sightings-Place-Index',
            Position=[lon_float, lat_float]
        )
        for result in response.get('Results', []):
            country = result['Place'].get('Country')
            if country:
                full_country_name = country_full_names.get(country.lower(), country.lower())
                return full_country_name
        return None
    except Exception as e:
        print(f"Error for lat: {lat}, lon: {lon}. Error: {e}")
        return None
            
# Fill in missing "country" values based on latitude and longitude
ufo.loc[ufo['country'].isnull(), 'country'] = ufo[ufo['country'].isnull()].apply(
    lambda row: get_full_country_name(row['latitude'], row['longitude']), axis=1)

# Check if there are any remaining missing values in the "country" column
ufo['country'] = ufo['country'].replace(country_code_mapping)  
ufo['country'] = ufo['country'].replace(country_full_names)  
ufo['UFO_shape'].fillna('unknown', inplace=True) 
ufo.to_csv('ufo_sightings_filled.csv', index=False)